### --- Day 13: Claw Contraption ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/13

In [2]:
#!import ../Utils.ipynb

In [3]:
using System.Text.RegularExpressions;

In [4]:
var inputLines = LoadPuzzleInput(2024, 13);
WriteLines(inputLines, maxRows: 7);

Loading puzzle file: Day13.txt
Total lines: 1279
Max line length: 23

Button A: X+25, Y+60
Button B: X+54, Y+25
Prize: X=12630, Y=3335

Button A: X+22, Y+58
Button B: X+65, Y+17
Prize: X=9383, Y=13439


In [5]:
string[] testInputLines = [
    "Button A: X+94, Y+34",
    "Button B: X+22, Y+67",
    "Prize: X=8400, Y=5400",
    "",
    "Button A: X+26, Y+66",
    "Button B: X+67, Y+21",
    "Prize: X=12748, Y=12176",
    "",
    "Button A: X+17, Y+86",
    "Button B: X+84, Y+37",
    "Prize: X=7870, Y=6450",
    "",
    "Button A: X+69, Y+23",
    "Button B: X+27, Y+71",
    "Prize: X=18641, Y=10279",
];

In [6]:
record Button(int Xboost, int Yboost) 
{
    public Button(string XBoost, string YBoost): this(int.Parse(XBoost), int.Parse(YBoost)) {}

    public Point AsPoint { get; } = new(Xboost, Yboost);
}
record Prize(int X, int Y)
{
    public Prize(string X, string Y) : this(int.Parse(X), int.Parse(Y)) {}

    public Point AsPoint { get; } = new(X, Y);
}
record Game(Button A, Button B, Prize Prize) {}

Regex buttonRegex = new(@"X\+(\d+), Y\+(\d+)");
Regex prizeRegex = new(@"X=(\d+), Y=(\d+)");

Game ParseGame(string[] inputLines)
{
    var (_, x, y) = buttonRegex.Match(inputLines[0]).Groups;
    Button aButton = new(x, y);
    (_, x, y) = buttonRegex.Match(inputLines[1]).Groups;
    Button bButton = new(x, y);
    (_, x, y) = prizeRegex.Match(inputLines[2]).Groups;
    Prize prize = new(x, y);

    return new(aButton, bButton, prize);
}

IList<Game> ParseAllGames(string[] inputLines)
{
    return inputLines.SeparateBy(line => line is "").Select(ParseGame).ToList();
}

There might be some clever mathematical way to solve this, but the basic brute-force approach also seems feasible, given the theoretical maximum of 100 x 100 combinations of A and B. This will probably come back to bite us for part 2, but let's see :)

In [7]:
// ...it costs 3 tokens to push the A button and 1 token to push the B button.
int GetTokenCost(int aPress, int bPress) => aPress * 3 + bPress;

// There's a maximum of 100 presses, but realistically we'll exceed our target long before 100
int PressLimit(int aBoost, int bBoost, int prize)
{
    int minBoost = Math.Min(aBoost, bBoost);
    return Math.Min(prize / minBoost, 100);
}

int SolveGame(Game game)
{
    var (aButton, bButton, prize) = game;
    
    int aPressLimit = PressLimit(aButton.Xboost, bButton.Xboost, prize.X);
    int bPressLimit = PressLimit(aButton.Yboost, bButton.Yboost, prize.Y);

    int minTokens = int.MaxValue;
    foreach (var aPresses in Enumerable.Range(1, aPressLimit))
    foreach (var bPresses in Enumerable.Range(1, bPressLimit))
    {
        var finalPosition = aButton.AsPoint * aPresses + bButton.AsPoint * bPresses;
        if (finalPosition == prize.AsPoint)
        {
            minTokens = GetTokenCost(aPresses, bPresses) switch {
                var cost when cost < minTokens => cost,
                _ => minTokens
            };
        }
    }
    return minTokens switch {
        < int.MaxValue => minTokens,
        _ => 0
    };
}

// Put it all together

int SolveAllGames(string[] inputLines) => ParseAllGames(inputLines).Select(SolveGame).Sum();

In [8]:
// So, the most prizes you could possibly win is two; the minimum tokens you
// would have to spend to win all (two) prizes is 480.

var testAnswer = SolveAllGames(testInputLines);
Console.WriteLine(testAnswer);

480


In [9]:
// Figure out how to win as many prizes as possible. What is the fewest tokens
// you would have to spend to win all possible prizes?

var part1Answer = SolveAllGames(inputLines);
Console.WriteLine(part1Answer);

35997


In [10]:
// 35997 is correct!
Ensure(35997, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/13

As suspected our basic approach for part 1 doesn't work here :) I played around with a few ideas regarding factorisation, but ultimately I had to take my first hint here!

The crutial hint here is that there aren't actually multiple solutions to the button combinations! If you consider A and B as vectors, and assuming they are not multiples of each other, you need a very specific number of A and B vectors to increment your way to the target. Changing the number of A / B vectors would change the resulting point away from the target.

As a simple example, let vector A point directly up, and vector B point directly right. There is only a single number of ups and rights that will let you hit the target, not multiple. All that stuff about "lowest cost" is a red herring!

Therefore, in part 2 we are basically solving 2 linear equations, which we can do with Gaussian Elimination (defined in [Utils](../Utils.ipynb)).

In [12]:
long SolveGame2(Game game)
{
    var (aButton, bButton, prize) = game;
    const decimal part2Extra = 10000000000000;

    decimal newPrizeX = prize.X + part2Extra;
    decimal newPrizeY = prize.Y + part2Extra;

    decimal[][] gameMatrix = [
        [aButton.Xboost, bButton.Xboost, newPrizeX],
        [aButton.Yboost, bButton.Yboost, newPrizeY]
    ];
    
    GaussianElim(gameMatrix);

    var (_, _, aPressesRaw) = gameMatrix[0];
    var (_, _, bPressesRaw) = gameMatrix[1];
    
    // If a solution is found, the presses should be integer values
    var aPresses = Math.Round(aPressesRaw, 0);
    var bPresses = Math.Round(bPressesRaw, 0);

    var xCheck = aButton.Xboost * aPresses + bButton.Xboost * bPresses;
    if (xCheck != newPrizeX)
    {
        // No solution for X
        return 0;
    }
    var yCheck = aButton.Yboost * aPresses + bButton.Yboost * bPresses;
    if (yCheck != newPrizeY)
    {
        // No solution for Y
        return 0;
    }

    // Solution found!
    return GetTokenCost(aPresses, bPresses);
}

long GetTokenCost(decimal aPress, decimal bPress) => (long)(aPress * 3 + bPress);

long SolveAllGames2(string[] inputLines) => ParseAllGames(inputLines).Select(SolveGame2).Sum();

In [13]:
var part2TestAnswer = SolveAllGames2(testInputLines);
Console.WriteLine(part2TestAnswer);

875318608908


In [14]:
// Using the corrected prize coordinates, figure out how to win as many prizes
// as possible. What is the fewest tokens you would have to spend to win all
// possible prizes?

var part2Answer = SolveAllGames2(inputLines);
Console.WriteLine(part2Answer);

82510994362072


In [15]:
// 82510994362072 is correct!
Ensure(82510994362072, part2Answer);